In [1]:
from datetime import date, time, datetime, timedelta
import numpy as np
import pandas as pd
from pandas import Series, DataFrame, Index
from IPython.display import Image

In [2]:
df_us = pd.read_csv("C:\Data\\HPI_master.csv",
                 parse_dates={'date_idx': [6,7]},
                 nrows=3080)

In [3]:
df_us.shape

(3080, 9)

In [4]:
df_us.set_index("date_idx", inplace=True, drop=False)

Indexing on the datetime column 'date' creates a 'datetime-aware' DateTimeIndex.  

In [5]:
df_us.set_index('place_name', inplace=True, drop=False)
df_us_plot = df_us.loc['United States']

Time series data lends itself well to plotting.  The bokeh package is used to plot the non-seasonal home price index for the entire U.S. using the 'df_us_plot' DataFrame created above

In [6]:
import bokeh.charts
import bokeh.charts.utils
import bokeh.io
import bokeh.models
import bokeh.palettes
import bokeh.plotting

# Display graphics in this notebook
bokeh.io.output_notebook()

Loading BokehJS ...

Plot the monthly aggregate home index values for the U.S. using the earliest and latest dates from the 'df_us_plot' DataFrame.

In [7]:
p = bokeh.charts.Line(df_us_plot, x='date_idx', y='index_nsa', color='firebrick', 
                      title="Monthly Aggregate Home Price Values in the U.S.")

# Display it
bokeh.io.show(p)

Create the DataFrame 'df_us_3' to select the rows with the values indicated below.

In [8]:
df_us_3 = df_us.loc[['West South Central Division', 'United States', 'Pacific Division']]

In [9]:
df_us_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 924 entries, West South Central Division to Pacific Division
Data columns (total 9 columns):
date_idx      924 non-null datetime64[ns]
hpi_type      924 non-null object
hpi_flavor    924 non-null object
frequency     924 non-null object
level         924 non-null object
place_name    924 non-null object
place_id      924 non-null object
index_nsa     924 non-null float64
index_sa      924 non-null float64
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 72.2+ KB


Plot the 3 regions using the U.S. home price index for comparison.

In [10]:
p = bokeh.charts.Line(df_us_3, x='date_idx', y='index_nsa', color='place_name', 
                      title="Monthly Home Price in West South Central and Pacific Division compared to U.S.",
                      legend="top_left")

bokeh.io.show(p)

In [11]:
df_states = pd.read_csv("C:\Data\\HPI_master.csv",                 
            skiprows=3082,
            usecols=(0, 1, 2, 3, 4, 5, 6, 7, 8),
            names=('hpi_type', 'hpi_flavor', 'frequency', 'level', 'place_name', 'place_id', 'yr', 'period', 'index_nsa'),
            header=None)

In [12]:
df_states["date_str"] = df_states['yr'].map(str) + 'Q' + df_states['period'].map(str)

In [13]:
df_states.iloc[0,-1]

'1986Q4'

Conver the 'date_str' column into a panda datetime column called 'date_idx'. 

In [14]:
df_states['date_idx'] = pd.to_datetime(df_states['date_str'])

Inspect the first 5 records in the 'df_states' DataFrame.

In [15]:
df_states.head()

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,date_str,date_idx
0,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1986,4,94.95,1986Q4,1986-10-01
1,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1987,1,100.91,1987Q1,1987-01-01
2,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1987,2,100.76,1987Q2,1987-04-01
3,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1987,3,94.02,1987Q3,1987-07-01
4,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1987,4,91.43,1987Q4,1987-10-01


In [16]:
df_states.describe(include=['O'])

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,date_str
count,96244,96244,96244,96244,96244,96244,96244
unique,4,3,1,4,464,464,166
top,traditional,all-transactions,quarterly,MSA,"Los Angeles-Long Beach-Glendale, CA (MSAD)",31084,2016Q2
freq,90806,67190,96244,69460,472,472,796


In [17]:
mask = (df_states['hpi_type'] == 'traditional') & (df_states['hpi_flavor'] == 'purchase-only') & \
       (df_states['level'] == 'State')

In [18]:
df_states = df_states.loc[mask]

In [19]:
df_states.shape

(5202, 11)

Before starting, review the organization of the 'df_us' DataFrame.  Notice the index currently in use is the column 'place_name'.

In [20]:
df_us.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3080 entries, East North Central Division to United States
Data columns (total 9 columns):
date_idx      3080 non-null datetime64[ns]
hpi_type      3080 non-null object
hpi_flavor    3080 non-null object
frequency     3080 non-null object
level         3080 non-null object
place_name    3080 non-null object
place_id      3080 non-null object
index_nsa     3080 non-null float64
index_sa      3080 non-null float64
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 240.6+ KB


Next, examine the quarterly date values from the 'df_states' DataFrame... 

In [21]:
 df_states.iloc[0:4, -1]

84584   1991-01-01
84585   1991-04-01
84586   1991-07-01
84587   1991-10-01
Name: date_idx, dtype: datetime64[ns]

...and compare with the monthly date values from the 'df_us' DataFrame.

In [22]:
df_us.iloc[0:4, -1]

place_name
East North Central Division    100.00
East North Central Division    101.11
East North Central Division    101.08
East North Central Division    101.10
Name: index_sa, dtype: float64

In [23]:
df_us.set_index('date_idx', inplace=True, drop=False)

In [24]:
df_us_qtr = df_us.resample('QS').mean()

#### 3. Create a column in the 'df_us' labeled 'place_name' with the value "U.S. Aggregate". 

In [25]:
df_us_qtr['place_name'] = 'U.S. Aggregate'

In [26]:
df_us_qtr.head()

,index_nsa,index_sa,place_name
date_idx,,,
1991-01-01,100.391000,100.357333,U.S. Aggregate
1991-04-01,100.923333,100.414333,U.S. Aggregate
1991-07-01,101.217667,100.475000,U.S. Aggregate
1991-10-01,101.875333,101.393667,U.S. Aggregate
1992-01-01,102.732333,102.703000,U.S. Aggregate


 Set the <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#Setting-and-resetting-Indicies">index</a> to the column 'place_name' and extract the rows using the <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#.loc-Indexer">.loc indexer</a>.

In [27]:
df_states.set_index('place_name', inplace=True, drop=False)

Create the mask using the boolean OR operator ( | ) 

In [28]:
mask = (df_states['place_name'] == 'Connecticut') | (df_states['place_name'] == 'District of Columbia')

Apply the mask using the <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#.loc-Indexer">.loc indexer</a> to create the 'hi_lo' DataFrame.

In [29]:
hi_lo = df_states.loc[mask]

In [30]:
hi_lo.head()

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,date_str,date_idx
place_name,,,,,,,,,,,
Connecticut,traditional,purchase-only,quarterly,State,Connecticut,CT,1991,1,100.00,1991Q1,1991-01-01
Connecticut,traditional,purchase-only,quarterly,State,Connecticut,CT,1991,2,97.81,1991Q2,1991-04-01
Connecticut,traditional,purchase-only,quarterly,State,Connecticut,CT,1991,3,97.11,1991Q3,1991-07-01
Connecticut,traditional,purchase-only,quarterly,State,Connecticut,CT,1991,4,96.53,1991Q4,1991-10-01
Connecticut,traditional,purchase-only,quarterly,State,Connecticut,CT,1992,1,97.30,1992Q1,1992-01-01


In [31]:
hi_lo.reset_index(drop=True, inplace=True)

In [32]:
hi_lo

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,date_str,date_idx
0,traditional,purchase-only,quarterly,State,Connecticut,CT,1991,1,100.00,1991Q1,1991-01-01
1,traditional,purchase-only,quarterly,State,Connecticut,CT,1991,2,97.81,1991Q2,1991-04-01
2,traditional,purchase-only,quarterly,State,Connecticut,CT,1991,3,97.11,1991Q3,1991-07-01
3,traditional,purchase-only,quarterly,State,Connecticut,CT,1991,4,96.53,1991Q4,1991-10-01
4,traditional,purchase-only,quarterly,State,Connecticut,CT,1992,1,97.30,1992Q1,1992-01-01
5,traditional,purchase-only,quarterly,State,Connecticut,CT,1992,2,95.18,1992Q2,1992-04-01
6,traditional,purchase-only,quarterly,State,Connecticut,CT,1992,3,95.07,1992Q3,1992-07-01
7,traditional,purchase-only,quarterly,State,Connecticut,CT,1992,4,96.03,1992Q4,1992-10-01
8,traditional,purchase-only,quarterly,State,Connecticut,CT,1993,1,92.43,1993Q1,1993-01-01
9,traditional,purchase-only,quarterly,State,Connecticut,CT,1993,2,91.63,1993Q2,1993-04-01


In [33]:
df_us_qtr.head()

,index_nsa,index_sa,place_name
date_idx,,,
1991-01-01,100.391000,100.357333,U.S. Aggregate
1991-04-01,100.923333,100.414333,U.S. Aggregate
1991-07-01,101.217667,100.475000,U.S. Aggregate
1991-10-01,101.875333,101.393667,U.S. Aggregate
1992-01-01,102.732333,102.703000,U.S. Aggregate


In [34]:
df_us_qtr.reset_index(inplace=True)

In [35]:
frames = [hi_lo, df_us_qtr]

In [36]:
plot_hi_low = pd.concat(frames)

In [37]:
plot_hi_low.shape

(307, 12)

In [38]:
#plot_hi_low.set_index('place_name', inplace=True, drop=False)

In [39]:
plot_hi_low

,date_idx,date_str,frequency,hpi_flavor,hpi_type,index_nsa,index_sa,level,period,place_id,place_name,yr
0,1991-01-01,1991Q1,quarterly,purchase-only,traditional,100.000000,NaN,State,1.0,CT,Connecticut,1991.0
1,1991-04-01,1991Q2,quarterly,purchase-only,traditional,97.810000,NaN,State,2.0,CT,Connecticut,1991.0
2,1991-07-01,1991Q3,quarterly,purchase-only,traditional,97.110000,NaN,State,3.0,CT,Connecticut,1991.0
3,1991-10-01,1991Q4,quarterly,purchase-only,traditional,96.530000,NaN,State,4.0,CT,Connecticut,1991.0
4,1992-01-01,1992Q1,quarterly,purchase-only,traditional,97.300000,NaN,State,1.0,CT,Connecticut,1992.0
5,1992-04-01,1992Q2,quarterly,purchase-only,traditional,95.180000,NaN,State,2.0,CT,Connecticut,1992.0
6,1992-07-01,1992Q3,quarterly,purchase-only,traditional,95.070000,NaN,State,3.0,CT,Connecticut,1992.0
7,1992-10-01,1992Q4,quarterly,purchase-only,traditional,96.030000,NaN,State,4.0,CT,Connecticut,1992.0
8,1993-01-01,1993Q1,quarterly,purchase-only,traditional,92.430000,NaN,State,1.0,CT,Connecticut,1993.0
9,1993-04-01,1993Q2,quarterly,purchase-only,traditional,91.630000,NaN,State,2.0,CT,Connecticut,1993.0


In [40]:
plot_hi_low.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307 entries, 0 to 102
Data columns (total 12 columns):
date_idx      307 non-null datetime64[ns]
date_str      204 non-null object
frequency     204 non-null object
hpi_flavor    204 non-null object
hpi_type      204 non-null object
index_nsa     307 non-null float64
index_sa      103 non-null float64
level         204 non-null object
period        204 non-null float64
place_id      204 non-null object
place_name    307 non-null object
yr            204 non-null float64
dtypes: datetime64[ns](1), float64(4), object(7)
memory usage: 31.2+ KB


In [41]:
plot_hi_low.place_name.unique()

array(['Connecticut', 'District of Columbia', 'U.S. Aggregate'], dtype=object)

In [42]:
p = bokeh.charts.Line(plot_hi_low, x='date_idx', y='index_nsa',  color='place_name', 
                     title= "Quarterly Home Price Values in the Washington Metro Area")

# Display it
bokeh.io.show(p)

Set the df_us DataFrame index to the column 'date_idx' using the <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#Setting-and-resetting-Indicies"> set_index() method </a>.

In [43]:
df_us_qtr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
date_idx      103 non-null datetime64[ns]
index_nsa     103 non-null float64
index_sa      103 non-null float64
place_name    103 non-null object
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 3.3+ KB


In [44]:
p = bokeh.charts.Line(df_us_qtr, x='date_idx', y='index_nsa', color='firebrick',  title="Quarterly Home Price Values in the U.S.")

# Display it
bokeh.io.show(p)

In [45]:
df_states.head()

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,date_str,date_idx
place_name,,,,,,,,,,,
Alaska,traditional,purchase-only,quarterly,State,Alaska,AK,1991,1,100.00,1991Q1,1991-01-01
Alaska,traditional,purchase-only,quarterly,State,Alaska,AK,1991,2,100.72,1991Q2,1991-04-01
Alaska,traditional,purchase-only,quarterly,State,Alaska,AK,1991,3,101.58,1991Q3,1991-07-01
Alaska,traditional,purchase-only,quarterly,State,Alaska,AK,1991,4,102.48,1991Q4,1991-10-01
Alaska,traditional,purchase-only,quarterly,State,Alaska,AK,1992,1,102.36,1992Q1,1992-01-01


Create the new DataFrame 'df_wash' to include row slices where the attribute .str.contains() value is "Washington" and the value "traditional" is in the column 'hpi_type'.

In [46]:
la

NameError: name 'la' is not defined

## Navigation

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/tree/master/"> Return to Chapter List </a>    